In [30]:
import SimpleITK as sitk
from os import walk
import numpy as np
import k3d, base64

In [14]:
data_directory = 'saline_bag/SE000002'
output_file = 'saline_bag/SE000002.mha'

Get all the files of the directory

In [22]:
f = []

for (dirpath, dirnames, filenames) in walk(data_directory):
    f.extend(filenames)
    
f = np.sort(f)

Find the DICOM files

In [24]:
print("Reading Dicom directory:", data_directory)
reader = sitk.ImageSeriesReader()
dicom_file_names = reader.GetGDCMSeriesFileNames(data_directory)

Reading Dicom directory: saline_bag/SE000002


Read the DICOM files

In [25]:
print("Reading the DICOM files in", data_directory)
reader.SetFileNames(dicom_file_names)
image = reader.Execute()

Reading the DICOM files in saline_bag/SE000002


In [26]:
size = image.GetSize()
print("Image size:", size[0], size[1], size[2])

Image size: 512 512 415


In [27]:
print("Writing image:", output_file)
sitk.WriteImage(image, output_file)

Writing image: saline_bag/SE000002.mha


Convert in a numpy array

In [ ]:
image_array = sitk.GetArrayFromImage(image) # z, y, x

Volume rendering

In [31]:
plot = k3d.plot()

# loading data to the `density` array
cmap = k3d.colormaps.matplotlib_color_maps.jet

size = np.array(image.GetSize()) * np.array(image.GetSpacing())

volume = k3d.volume(
    image_array.astype(np.float32),
    alpha_coef=5,
    shadow='dynamic',
    samples=300,
    shadow_res=128,
    shadow_delay=50,
    color_range=[0.01, max(mu_cube, mu_cylinder)],
    color_map=(np.array(cmap).reshape(-1,4)).astype(np.float32)
)

volume.transform.bounds = [-size[0] / 2, size[0] / 2,
                           -size[1] / 2, size[1] / 2,
                           -size[2] / 2, size[2] / 2]

plot = k3d.plot(camera_auto_fit=True, height=400)

plot += volume
plot.display()

NameError: name 'mu_cube' is not defined

In [ ]:
plot.fetch_screenshot()

data = base64.b64decode(plot.screenshot)
with open('plots/simpleCT_volume_rendering.png','wb') as fp:
    fp.write(data)